# Portfolioprojekt: Depression and anxiety data  (EDA-file)

The project was given from:

https://www.kaggle.com/datasets/shahzadahmad0402/depression-and-anxiety-data/code


***
We have the following tasks:
* Data reading and cleaning,
* first Feature Engineering,
* Base model, fit the dates to the model,
* Combination to the pipeline,
* Model interpretation.
***

The data-set is given with th columns

| **Column** | **Description** |
| ------------ | :-----------------: |
| id | each number is a participant in the experiment |
| school_year | years in school |
| age | |
| gender | |
| bmi | body mass index |
| who_bmi | bmi category |
| phq_score | measure the severity of symptoms related to depression, anxiety, and other related disorders in patients |
| depression_severity | degree or intensity of symptoms experienced by an individual with depression |
| depressiveness | |
| suicidal | the candidate have suicide thought |
| depression_diagnosis | the candidate already have depression diagnosis |
| depression_treatment | the candidate already have depression treatment |
| gad_score | measure that assesses the severity of Generalized Anxiety Disorder |
| anxiety_severity |  intensity of symptoms experienced by an individual with anxiety |
| anxiousness | |
| anxiety_diagnosis | the candidate already have anxiety diagnosis |
| anxiety_treatment | the candidate already have anxiety treatment |
| epworth_score |  score to assess daytime sleepiness ytime sleepiness |
| sleepiness | |

# EDA in Power-BI:
We saw, that the column: pdh_score is very strong related to depressiveness (means: score >= 9 is depressive, score < 9 no depressive), further the depression_severity is also strong related to the depressiveness, such that we skip this 2 columns

**Szenario:** 
Based on the target 'depressivenes', we apply the EDA and a first base-model with logging and grid-search.

We deal with a given csv-file and split it into a train + test dataset to apply the fit and prediction.
Further, we have a data-cleaning, which ste the correct types and cleaned with NaN entries.

## Preparation

We solve the following problem: Prediction of depressiveness given with important featurs: This is done by
* Preparation of the dataset
* Problem is based as a classification problem, which can be done with logistic regression, random forest, KNN, decision-tree
* We apply the model to an aim-data set and predit, if a person has a tendency to depression or not
* We like to built an app for a clinical application or an application for a Universities to see, if their students have some tendencises
* We apply given datasets of a clinical study

### Define Metric

We apply the F1-matric to get a balance between the precision and recall.

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, precision_score


# Metrics considerations

# Accuracy: The proportion of correct predictions. 
            # In your case this metric might be less important, 
           # I may have an unbalanced dataset (more good purchases than bad purchases).
# Better metrics
# Recall (Sensitivity): The proportion of correctly identified bad purchases out of all 
# actual bad purchases. Here I optimize: Identify as many bad purchases as possible.
# Precision: The proportion of correctly identified bad purchases out of all as bad purchases 
# predicted cars. Important to minimize false purchases.
# It is best to weight between the two:
# F1-Score: The harmonic mean of precision and recall. 
# Here I have to find a balance between precision and recall.

# We apply the F1-score

### Gather Data

We have the dataset: *depression_anxiety_data.csv*. With the target-column: 'depressiveness

In [ ]:
# import modules 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# Modelle
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Pipeline
from sklearn.pipeline import Pipeline
# Columntransormer
from sklearn.compose import ColumnTransformer
# OneHotEndode,, Scaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Train/Test Splitt, Crossvaue-Score, SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures

In [ ]:

# read data (start from the dataset)
df = pd.read_csv("data/depression_anxiety_data.csv")

display(df.head())
#print(df)

## EDA

### Understand Data

Mach dich zu Beginn mit den Daten vertraut, damit du später bei der Datenbereinigung und -aufbereitung weißt, worauf du achten solltest.

**Tipp**: Die Spalte `'PurchDate'` ist als Unix-Timestamp in ganzen Sekunden angegeben. Um diese Spalte mit `pandas` in ein Datumsformat umzuwandeln, kannst du folgenden Code nutzen: `my_df.loc[:, 'PurchDate'] = pd.to_datetime(my_df.loc[:, 'PurchDate'], unit='s')`

In [ ]:


# 1. See the datatypes
print("\nDatatypes of the column:")
display(df.dtypes)

# 2. Statistical description

print("\n Statistical Description:")
display(df.describe())


In [ ]:
#check NaNs and duplicates
print('Index')
print('index_size', df.index.size)
print('Columns with NaN')
print('is NaN', df.isna().sum())
print('Duplicates in Columns')
print('duplicated', df.duplicated().sum())
#note: no NaNs, no duplicates, no cleaning required

In [ ]:
# Exists duplicates
duplicates = df[df.duplicated()]
print("Show duplicates:")
print(duplicates)

# Clean duplicates
df_clean = df.drop_duplicates()

# Optional: Do we have duplicates
print("Overview, if duplicates after cleaning:")
print('duplicated', df_clean.duplicated().sum())


In [ ]:
# Consider the important columns
columns_of_interest = [
    'gender', 'who_bmi', 'depression_severity', 'depressiveness', 'suicidal', 
    'depression_diagnosis', 'depression_treatment', 'anxiety_severity', 
    'anxiousness', 'anxiety_diagnosis', 'anxiety_treatment', 'epworth_score', 
    'sleepiness'
]

# Show the unique values of each column
for column in columns_of_interest:
    unique_values = df[column].unique()
    print(f"Unique columns '{column}':")
    print(unique_values)
    print()




In [ ]:
# Modells with targets and features

# Target columns (later we put 'depression_treatment', 'anxiety_treatment' together to 'treatment_status')

target_cols =['anxiousness', 'depressiveness', 'depression_treatment', 'anxiety_treatment', 'suicidal']

# first all numerical columns

num_cols =  ['school_year', 'age', 'bmi', 'phq_score', 'gad_score', 'epworth_score' ]


# categorical columns, which can be transformed simple to numerical columns (only true/false entires)

cat_cols_trans = ['gender', 'depression_diagnosis', 'depression_treatment',  'anxiety_diagnosis', 'sleepiness']


# We have 3 catergoical columns with more that 2 entries:

cat_cols = ['who_bmi', 'anxiety_severity', 'depressiv_severity']


In [ ]:
# Clean the Datatypes:


# Drop all NaNs (we have ony a few NaNs in the columns): 
df = df.dropna()


# Correct Datatypes of the target:
# and the feature gender (both int)

df.gender = df.gender.map({'male':1, 'female':0})


# Define the targets:
#1.) anxiousness:  Prediction:  0: False (no anxious) / 1: True (anxious)
#2.) depressiveness: Prediction: 0: False (no depression) / 1: True (depression)
#3.) will_get_treatment: Prediction: 0: False (no get-treatement) / 1: True (get-treatment)
#4.) suicidality: Prediction: 0: False (no suicidality) / 1: True (suicidality)

df['anxiousness'] = df['anxiousness'].astype(int)
df['depressiveness'] = df['depressiveness'].astype(int)

df['treatment_status'] = df['depression_treatment'] | df['anxiety_treatment']
df['treatment_status'] = df['treatment_status'].astype(int)


df['suicidal'] = df['suicidal'].astype(int)

# Change the binary categorial feature variables into numerical 

# Convert binary categorical features to integer

# List of categorical columns with binary values that need to be converted to integers
cat_cols_trans = ['gender', 'depression_diagnosis', 'depression_treatment',  
                      'anxiety_diagnosis', 'sleepiness']

df[cat_cols_trans] = df[cat_cols_trans].astype(int)


df['anxiety_treatment'] = df['anxiety_treatment'].astype(int)

# Describe the dates
display(df.describe())

In [ ]:

# Durch jede Spalte iterieren und die Verteilung der Werte anzeigen
for column in columns_of_interest:
    value_counts = df[column].value_counts()
    print(f"Verteilung der Werte in der Spalte '{column}':")
    print(value_counts)
    print()

In [ ]:
# Clean the data and correctionof the types

print("\nDatentypen der Spalten:")
display(df.dtypes)

  

In [ ]:
# check distribution of values in each feature
# Solution:
display(df.describe())
display(sns.pairplot(df))  # needs time

In [ ]:
import seaborn as sns



# critical features, we skip to predict the 'depressiveness':   'phq_score', 'depression_severity'
num_cols = ['school_year', 'age', 'gender', 'bmi', 'depression_diagnosis', 'gad_score',
            'anxiety_diagnosis', 'epworth_score', 'sleepiness']



targets = ['anxiousness', 'depressiveness', 'treatment_status', 'suicidal']

# (depression_treatment and anxiety_treatment  are in the target treatment_status)

cat_cols = ['who_bmi', 'anxiety_severity']

# Selected columns
selected_columns = num_cols

# Korrelationsmatrix der ausgewählten Spalten
corr_matrix = df[selected_columns].corr()


# Heatmap der Korrelationsmatrix
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin = -1, vmax = 1, fmt='.2f')
plt.title('Korrelationsmatrix')
plt.show()


In [ ]:



num_cols = ['school_year', 'age', 'gender', 'bmi', 'depression_diagnosis', 'gad_score',
            'anxiety_diagnosis', 'epworth_score', 'sleepiness']


num_cols__no_pca = ['gender', 'bmi']

num_cols_pca = ['school_year', 'age', 'phq_score', 'depression_diagnosis', 'gad_score',
            'anxiety_diagnosis', 'epworth_score', 'sleepiness']

targets = ['anxiousness', 'depressiveness', 'treatment_status', 'suicidal']

# (depression_treatment and anxiety_treatment  are in the target treatment_status)

cat_cols = ['who_bmi', 'anxiety_severity']

In [ ]:

# Histogram and boxplot for the numerical features with the target "depressiveness" as Hue

for col in num_cols:

    # Full plot histogram and boxplot

    fig, axes = plt.subplots(2, 1, figsize=(20, 6), sharex=True, gridspec_kw={'height_ratios': [5, 1]})
    
    # Histogram
    sns.histplot(data=df_clean, x=col, hue='depressiveness', kde=True, multiple="stack", ax=axes[0])
    axes[0].set_title(f'Histogram of {col} by depressiveness')
    
    # Boxplot
    sns.boxplot(data=df_clean, x=col, hue='depressiveness', ax=axes[1])
    axes[1].set_title(f'Boxplot of {col} by depressiveness')



    # Titles of the axis
    axes[1].set_xlabel(col)
    axes[1].set_ylabel('')
    plt.show()
    
        

In [ ]:
display(df.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cat_columns = ['who_bmi', 'anxiety_severity']
hue_column = 'depressiveness'

# Bar- and Pis-plot for all the categorical features, based on the hue "depressiveness"
for col in cat_columns:
    # Barplot 
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=col, hue=hue_column)
    plt.title(f'Distrution of {col} with {hue_column}')
    plt.xlabel(col)
    plt.ylabel('Number')
    plt.xticks(rotation=0)
    plt.show()

    # Pieplo

    hue_levels = df[hue_column].unique()
    for level in hue_levels:
        counts = df[df[hue_column] == level][col].value_counts()

        plt.figure(figsize=(8, 8))
        counts.plot(kind='pie', autopct='%1.1f%%')
        plt.title(f'Procetual distribution of the {col} for {hue_column} = {level}')
        plt.ylabel('')
        plt.show()


In [ ]:
cat_aim = ['depressiveness']

# Bar- and Pie-Plots for the target
for col in cat_aim:
    # Daten für den Plot vorbereiten
    counts = df[col].value_counts()

    # Barplot
    plt.figure(figsize=(10, 6))
    counts.plot(kind='bar')
    plt.title(f'Distribution of the {col}')
    plt.xlabel(col)
    plt.ylabel('Anzahl')
    plt.xticks(rotation=0)
    plt.show()

    # Pieplot
    plt.figure(figsize=(8, 8))
    counts.plot(kind='pie', autopct='%1.1f%%')
    plt.title(f'Precentage Distribution of the columns {col}')
    plt.ylabel('')
    plt.show()

In [ ]:
# Important Columns and their relation, e.g., for the PCA

#num_cols = ['school_year', 'age', 'gender', 'bmi', 'phq_score', 'depression_diagnosis', 'gad_score',
#            'anxiety_diagnosis', 'epworth_score', 'sleepiness']

num_cols = ['school_year', 'age', 'gender', 'bmi', 'phq_score', 'gad_score',
            'anxiety_diagnosis', 'epworth_score', 'sleepiness']

num_cols_no_pca = ['gender', 'bmi']

num_cols_pca = ['school_year', 'age', 'depression_diagnosis', 'gad_score',
            'anxiety_diagnosis', 'epworth_score', 'sleepiness']

targets = ['anxiousness', 'depressiveness', 'treatment_status', 'suicidal']

# (depression_treatment and anxiety_treatment  are columns, which are used in the target treatment_status)

#cat_cols = ['who_bmi', 'depression_severity', 'anxiety_severity']
cat_cols = ['who_bmi', 'anxiety_severity']

target = ['depressiveness']

In [ ]:
# Important numerical features:
#
num_cols_import = num_cols_no_pca + num_cols_pca

# Pairwise plot for the numerical columns Plot give with the target depressiveness
sns.pairplot(df_clean[num_cols_import + target], hue='depressiveness')
plt.show()


## Train-Test-Split

Splitt of the train and test-dates

**Always fit on Train Set only!** 

In [ ]:
# read data (start from the dataset)
df = pd.read_csv("data/depression_anxiety_data.csv")


# perform train-test-split


# Feature-Matrix und Zielvektor definieren
features = df.drop(columns=['depressiveness'])
target = df['depressiveness']

# Daten aufteilen in Trainings- und Testdaten
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, test_size=0.1, random_state=42)


In [ ]:
display(features_train)
display(target_train)

In [ ]:
# save features_test as 'features_test.csv'

# Testdaten speichern
# features_test.to_csv('features_test.csv', index=False)


# I first clean the data-set, while we have in the target of the test also NaN, therefore I clean all and save ist after that!

## Data Preparation


We apply the *Data Preparation*, e.g., clean the dates and transprm the dates, e.g., special data-types

### Datatype Transformation

In [ ]:
# Data Cleaning and Transformation Function
def clean_data(X, y=None):
    """Cleans and transforms the input feature DataFrame and optionally the target.

    This function performs the following operations on the feature DataFrame (X):
        - Drops rows with any missing values (NaNs) in either X or y to ensure they remain aligned.
        - Converts binary categorical features into numerical values (0 and 1).
        - Optionally, cleans and transforms the target DataFrame (y) if provided.
        - Converts datetime features to the appropriate datetime format.
        - Returns the cleaned features and optionally the cleaned target.

    Args: 
        X (pd.DataFrame): The feature data.
        y (pd.DataFrame, optional): The target data. Default is None.
        
    Returns:
        pd.DataFrame: A cleaned DataFrame with transformed features.
        pd.DataFrame: Optionally, the cleaned target DataFrame if y is provided.
    """
    # Concatenate X and y to handle missing values across both simultaneously
    if y is not None:
        # Combine X and y into a single DataFrame for joint NaN removal
        combined = pd.concat([X, y], axis=1)
        
        # Drop rows with any missing values in either X or y
        combined_clean = combined.dropna()
        
        # Separate X and y again
        X_clean = combined_clean.iloc[:, :X.shape[1]]
        y_clean = combined_clean.iloc[:, X.shape[1]:]
    else:
        # If y is not provided, only clean X
        X_clean = X.dropna()

    # List of categorical columns with binary values that need to be converted to integers
    cat_cols_trans = ['gender', 'depression_diagnosis', 'depression_treatment',  
                      'anxiety_diagnosis', 'sleepiness']

    # Map 'gender' column to integers: 'male' -> 1, 'female' -> 0
    X_clean['gender'] = X_clean['gender'].map({'male': 1, 'female': 0})

    # Convert the specified binary categorical columns in X to integers (0 and 1)
    X_clean[cat_cols_trans] = X_clean[cat_cols_trans].astype(int)

    if y is not None:
        # Convert all target variables to integers
        y_clean = y_clean.astype(int)
        # Return both cleaned X and cleaned y
        return X_clean, y_clean
    else:
        # If y is not provided, return only the cleaned features
        return X_clean


In [ ]:
features_train, target_train = clean_data(features_train, target_train)

display(features_train.head())
display(target_train.head())

In [ ]:
# Preparation of the test-data-set
features_test, target_test = clean_data(features_test, target_test)

display(features_test.head())
display(target_test.head())

# save features_test as 'features_test.csv'

# Testdaten speichern
features_test.to_csv('features_test.csv', index=False)

### Data Imputation


#It is not necessary while we have a very simple data-set, where all the dates are given

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer


print('Columns with NaN')
print('is NaN', features_train.isna().sum())

# Wir übernehmen den KNN Imputer:



# Possible imputer if necessary
""" num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('std_scaler', StandardScaler())
])

# Fit and Transform the training-data
features_train_transformed = num_transformer.fit_transform(features_train)

# fit and transorm the test-data
features_test_transformed = num_transformer.transform(features_test)

# Show the results
print("Transformed training-dates:\n", features_train_transformed)
print("Transformed testdates:\n", features_test_transformed) """



In [ ]:
# Pipeline for the imputer

from sklearn.impute import SimpleImputer


num_cols = ['school_year', 'age', 'gender', 'bmi', 'phq_score', 'depression_diagnosis', 'gad_score',
            'anxiety_diagnosis', 'epworth_score', 'sleepiness']

num_cols_no_pca = ['gender', 'bmi']

num_cols_pca = ['school_year', 'age', 'phq_score', 'depression_diagnosis', 'gad_score',
            'anxiety_diagnosis', 'epworth_score', 'sleepiness']

targets = ['anxiousness', 'depressiveness', 'treatment_status', 'suicidal']

cat_cols = ['who_bmi', 'depression_severity', 'anxiety_severity']





cat_pipe = Pipeline(steps=[
    ("imp", SimpleImputer(strategy="constant", fill_value="missing")),
    ("ohe", OneHotEncoder(handle_unknown='ignore')),
])




#num_pipe = Pipeline(steps=[
#    ("imp", SimpleImputer(strategy="mean")),
#])

num_pipe = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('std_scaler', StandardScaler())
])




# Warnung vermeiden durch Kopieren des DataFrames, sher wichtig sonst funktioniert es nicht!
features_train = features_train.copy()
target_train = target_train.copy()


# Spalten-Transformator
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipe, num_cols),
        ('cat', cat_pipe, cat_cols)
    ])

# Erstellen einer kompletten Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Fitte die Pipeline auf die Trainingsdaten
features_train_transformed = pipeline.fit_transform(features_train)

# Transformiere die Testdaten
features_test_transformed = pipeline.transform(features_test)

# Ergebnisse anzeigen
print("Transformierte Trainingsdaten:\n", features_train_transformed)
print("Transformierte Testdaten:\n", features_test_transformed)


# Überprüfen auf NaN-Werte
print('Anzahl der NaN-Werte in den transformierten Trainingsdaten:', np.isnan(features_train_transformed).sum())
print('Anzahl der NaN-Werte in den transformierten Testdaten:', np.isnan(features_test_transformed).sum())

### Deal with outliers

In [ ]:
# Distribution of the datasets
print(target_train.value_counts())

# Optional: precentage distribution of the dataset
print(target_train.value_counts(normalize=True))

We apply the DBScan to detect the outliers. We saw, that we have only few outliers, which can be important and do not clean them.

In [ ]:
from scipy.stats import pearsonr
from sklearn.cluster import DBSCAN


def identify_outliers_dbscan(X_train, correlation_threshold, num_cols, eps=0.5, min_samples=5, outlier_percentage=0.1):
    outlier_mask = pd.Series([False] * X_train.shape[0], index=X_train.index)
    columns = num_cols
    all_outlier_indices = []
    
    for i in range(len(columns)):
        for j in range(i + 1, len(columns)):
            col_x = columns[i]
            col_y = columns[j]
            
            # Berechnen des Korrelationskoeffizienten
            correlation, _ = pearsonr(X_train[col_x], X_train[col_y])
            
            # Anwenden von DBSCAN nur bei hoher Korrelation
            if abs(correlation) >= correlation_threshold:
                X = X_train[[col_x, col_y]].values
                
                # DBSCAN anwenden
                dbscan = DBSCAN(eps=eps, min_samples=min_samples)
                dbscan.fit(X)
                labels = dbscan.labels_
                
                # Ausreißer sind die Punkte mit dem Label -1
                outliers = labels == -1
                all_outlier_indices.extend(X_train.index[outliers])
    
    # Begrenzen der Anzahl der Ausreißer auf den gewünschten Prozentsatz
    all_outlier_indices = np.unique(all_outlier_indices)
    num_outliers = int(len(X_train) * outlier_percentage)
    
    if len(all_outlier_indices) > num_outliers:
        np.random.seed(0)  # Für Wiederholbarkeit
        selected_outliers = np.random.choice(all_outlier_indices, num_outliers, replace=False)
    else:
        selected_outliers = all_outlier_indices
    
    outlier_mask.loc[selected_outliers] = True
    
    return outlier_mask


# important Columns:

num_cols_import = ['school_year', 'age', 'gender', 'bmi', 'phq_score', 'depression_diagnosis', 'gad_score',
            'anxiety_diagnosis', 'epworth_score', 'sleepiness']
# Anwenden von identify_outliers_dbscan und Begrenzen auf 10% der Top Outliers
outlier_mask = identify_outliers_dbscan(features_train, correlation_threshold=0.7, 
                                        num_cols=num_cols_import, eps=0.7, min_samples=3, outlier_percentage=0.1)



In [ ]:
num_cols_print = num_cols_import





# Plot der Daten mit Ausreißern als Hue
def plot_data_with_outliers(X_train, outlier_mask, title):
    X_train_copy = X_train.copy()
    X_train_copy['Outlier'] = outlier_mask
    sns.pairplot(X_train_copy, hue='Outlier', vars=num_cols_print, diag_kind='kde', palette={True: "red", False: "blue"})
    plt.suptitle(title)
    plt.show()

plot_data_with_outliers(features_train, outlier_mask, "Daten mit Ausreißern markiert")

# Entfernen der Ausreißer
features_train_no_outliers = features_train[~outlier_mask]

# Um sicherzustellen, dass keine Spalten mit 0 Varianz vorhanden sind, können Sie diese vor dem Plotten überprüfen
def plot_data_no_outliers(X_train, num_cols_print, title):
    for col in num_cols:
        if X_train[col].nunique() <= 1:
            print(f"Skipping {col} as it has 0 variance")
            continue
        plt.figure(figsize=(10, 6))
        sns.histplot(data=X_train, x=col, hue='depressiveness', kde=True, multiple="stack")
        plt.title(f'Histogram of {col} by depressiveness')
        plt.show()

plot_data_with_outliers(features_train_no_outliers, outlier_mask[~outlier_mask], "Daten nach Entfernen der Ausreißer")

In [ ]:
# Number of outliers
num_outliers = outlier_mask.sum()
print(f"Number of Outliers: {num_outliers}")


### Resample

We apply an oversampler to balance the target colum.

*resampling* uses the `imblearn.pipeline`.

In [ ]:
# We apply the resampling in the pipeline

from imblearn.over_sampling import RandomOverSampler
from sklearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import RandomForestClassifier

# We apply an oversampler
#rf_pipe = ImbPipeline(steps=[
#    ("pre", preprocessing),
#    ("oversampler", RandomOverSampler()),   # Hinzufügen des RandomOverSamplers
#    ("kbest", SelectKBest(k=10, score_func=f_classif)),
#    ("model", RandomForestClassifier()),
#])


In [ ]:
# We see the unbalance of the target and apply it to the pipeline
print(target_train.value_counts())

# Percantage of the imbalance
print(target_train.value_counts(normalize=True))





In [ ]:
display(features_train)

display(features_train.dtypes)


display(target_train)

display(target_train.dtypes)

## Modeling

Kommen wir nun zur Kernaufgabe eines *Data Scientist*, dem Bauen von Maschine-Learning-Modellen. Wie du es im Training bereits gelernt hast, ist das Optimieren eines Modells ein iterativer Prozess. Die nachfolgenden Schritte wirst du also sicher mehr als einmal durchführen.

### Build a simple Baselinemodel

We built a baseline model:

1. Numerical features (`num_cols`) and categorical features (`cat_cols`),
3. We apply a baseline model,
3. We apply a pipeline (`model_baseline`),
4. We apply the `features_train` and the `target _train` for the  *fitting* ,
5. We apply the `model_baseline` to predict the `features_test`.

In [ ]:
# define num_cols and cat_cols



num_cols = ['school_year', 'age', 'gender', 'bmi', 'phq_score', 'depression_diagnosis', 'gad_score',
            'anxiety_diagnosis', 'epworth_score', 'sleepiness']

num_cols_no_pca = ['gender', 'bmi']

num_cols_pca = ['school_year', 'age', 'phq_score', 'depression_diagnosis', 'gad_score',
            'anxiety_diagnosis', 'epworth_score', 'sleepiness']

#targets = ['anxiousness', 'depressiveness', 'treatment_status', 'suicidal']

cat_cols = ['who_bmi', 'depression_severity', 'anxiety_severity']




target = ['depressiveness']




# Definieren der Spaltenlisten für die depressiveness


#num_2_cols = ['school_year', 'age', 'gender', 'bmi', 'phd_score', 'depression_diagnosis', 'gad_score',
#            'anxiety_diagnosis', 'epworth_score', 'sleepiness']

num_2_cols = ['school_year', 'age', 'gender', 'bmi', 'depression_diagnosis', 'gad_score',
            'anxiety_diagnosis', 'epworth_score', 'sleepiness']

In [ ]:
display(target_train)

In [ ]:

# categorical columns
cat_2_cols = ['who_bmi', 'depression_severity', 'anxiety_severity']
cat_2_cols = ['who_bmi', 'anxiety_severity']


# number of categories for each column
for col in cat_2_cols:
    num_categories = df[col].nunique()
    print(f'Number of cattegories in {col}: {num_categories}')
    


In [ ]:
# instantiate model

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier





In [ ]:
# Installation des TargetEncoder von category_encoder

!pip install category_encoders
import category_encoders as ce

In [ ]:
for col in cat_2_cols:
    features_train[col] = features_train[col].astype('category')

In [ ]:


cat_2_cols = ['anxiety_severity','who_bmi']

# Ensure target_train is a numpy array or a pandas Series
target_train = np.array(target_train)


numeric_features = num_2_cols
categorical_features = cat_2_cols

# Preprocessing Pipelines für numerische und kategorische Daten
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('encoder', ce.TargetEncoder())])

# ColumnTransformer zum Kombinieren der Preprocessing-Schritte
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        ])

# Erstellen der Pipeline mit Preprocessing und dem Modell
rf_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())])

# Fit auf die Trainingsdaten
rf_pipe.fit(features_train, target_train)

In [ ]:





# Eigene Klasse mit Zeiterfassung



cat_pipe = Pipeline(steps=[
    ("imp", SimpleImputer(strategy="constant", fill_value="missing")),
    ("target_enc", ce.TargetEncoder())  # TargetEncoder hinzufügen
#    ("ohe", OneHotEncoder(handle_unknown='ignore')),  # One-Hot encoder
])

num_pipe = Pipeline(steps=[
    ("imp", SimpleImputer(strategy="mean")),
#    ("poly", PolynomialFeatures(degree=2, include_bias=False)),  # PolynomialFeatures hinzufügen, bringt nichts, da man
#    Lineare Kombinationen hat
    ("scaler", StandardScaler()),
])


preprocessing = ColumnTransformer(
    transformers=[
        ("num_trans", num_pipe, num_2_cols),
        ("cat_trans", cat_pipe, cat_2_cols),
    ],
    remainder = "drop",
)


# Resampling

from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler


# In meine Pipeline habe ich den RandomOverSapler eingebout 


rf_pipe = ImbPipeline(steps=[
    ("oversampler", RandomOverSampler()),   # Hinzufügen des RandomOverSamplers
    ("pre", preprocessing),
    ("kbest", SelectKBest(k=10, score_func=f_classif)),
    ("model", RandomForestClassifier()),
#    ("model", RandomForestClassifier_mod()),
])



In [ ]:
# fit pipeline on cleaned (and filtered) training set

rf_pipe.fit(features_train, target_train)  # Standard

# rf_pipe_mod.fit(features_train, target_train)  # Modified

In [ ]:
# predict and evaluate on test set

print(classification_report(target_test, rf_pipe.predict(features_test)))






In [ ]:
import logging
import os

# Manually close the current log handlers
logging.shutdown()

In [ ]:
import logging
import os

# Ensure the 'log' directory exists
log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Confirm that the directory was created
if os.path.exists(log_dir):
    print(f"Directory '{log_dir}' exists or was successfully created.")
else:
    print(f"Failed to create directory '{log_dir}'.")

# Check the current working directory
print(f"Current working directory: {os.getcwd()}")

# Configure logging explicitly
log_file_path = os.path.join(log_dir, 'model_evaluation.log')
print(f"Log file path: {log_file_path}")  # This should point to the 'log' directory

# Create and configure the logger explicitly
logger = logging.getLogger('model_evaluation_logger')
logger.setLevel(logging.INFO)


# Avoid duplicate logs by clearing existing handlers if any

if logger.hasHandlers():

    logger.handlers.clear()


# Create a file handler for logging

file_handler = logging.FileHandler(log_file_path)

file_handler.setLevel(logging.INFO)


# Create a logging format

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

file_handler.setFormatter(formatter)


# Add the file handler to the logger

logger.addHandler(file_handler)


# Example log entry

logger.info("Logging to the log/model_evaluation.log file.")

print("Logging setup completed.")



In [ ]:
from sklearn.metrics import classification_report

# Model name
model_name = type(rf_pipe.named_steps['model']).__name__

# Model parameters
model_params = rf_pipe.named_steps['model'].get_params()

# Classification report
classification_rep = classification_report(target_test, rf_pipe.predict(features_test))

# Log the model details
logger.info("Model Evaluation")
logger.info(f"Model Name: {model_name}")
logger.info("Model Parameters:")
for param, value in model_params.items():
    logging.info(f"{param}: {value}")

# Log the classification report
logger.info("Classification Report:")
logger.info("\n" + classification_rep)

print(f"Model details and evaluation have been logged to 'log\model_evaluation.log'.")


In [ ]:
from sklearn.model_selection import cross_val_score


# Assuming 'model' is your trained model and 'X' and 'y' are your features and target
scores = cross_val_score(rf_pipe, features_train, target_train, cv=5, scoring='accuracy')
print("Cross-Validation Accuracy Scores:", scores)
print("Mean Cross-Validation Accuracy:", scores.mean())

In [ ]:
# Prediction

print(rf_pipe.predict(features_test))

In [ ]:

# Feature Importance ermitteln
rf_model = rf_pipe.named_steps['model']

# Extrahieren der Feature-Namen nach der Transformation
preprocessor = rf_pipe.named_steps['pre']
kbest = rf_pipe.named_steps['kbest']
selected_features = kbest.get_support(indices=True)

# Numerische und kategorische Feature-Namen nach der Transformation
num_feature_names = num_2_cols
cat_feature_names = cat_2_cols

# Kombinierte Liste der ursprünglichen Features
all_features = num_feature_names + cat_feature_names

# Ausgewählte Features nach SelectKBest
selected_feature_names = [all_features[i] for i in selected_features]

# Erzeuge eine Serie mit den Feature Importances
feature_importance_rf = pd.Series(rf_model.feature_importances_, index=selected_feature_names)

# Sortiere die Werte und wähle die Top 10 Features aus
top_10_features = feature_importance_rf.nlargest(10)

# Erzeuge ein Balkendiagramm
plt.style.use('fivethirtyeight')
top_10_features.plot(kind='barh', figsize=(10, 8))

# Füge Beschriftungen hinzu
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('Feature Importance in Random Forest')

# Zeige das Diagramm an
plt.show()

In [ ]:
print(all_features)

In [ ]:

# Sortiere die Werte und wähle die Top 10 Features aus
top_10_features = feature_importance_rf.nlargest(10).sort_values()

# Erzeuge ein Balkendiagramm für die Top 10 Features
plt.style.use('fivethirtyeight')
top_10_features.plot(kind='barh', figsize=(10, 8))

# Füge Beschriftungen hinzu
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('Top 10 Feature Importance in Random Forest')

# Zeige das Diagramm an
plt.show()

In [ ]:



from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier




# Fist Idea of a grid search:

# Modified GridSearch

#class GridSearchCV_mod(GridSearchCV):
#    @timeit
#    def fit(self, X, y=None, **fit_params):
#        return super().fit(X, y, **fit_params)




# Parameter Grid definieren für RF_mod
#param_grid = {
#    'model__n_estimators': [100],  # , 200],  # , 300],
#    'model__max_features': [None, 'sqrt'],  # , 'log2'],
#   'model__max_depth': [10, 20],  # , 30, None],
#    'model__min_samples_split': [2, 5],  # , 10],
#    'model__min_samples_leaf': [2, 4],  # [1, 2, 4]
#    'model__max_samples_leaf': [0.1, 0.2],  # Beispielsweise zwei Werte zur Auswahl
#}

# Parameterraum mit normalem RF
#param_grid = {
#    'model__n_estimators': [100] , # , 200],  # , 300],
#    'model__max_features': [None, 'sqrt'],  # , 'log2'],
#    'model__max_depth': [10, 20],  # , 30, None],
#    'model__min_samples_split': [2, 5],  # , 10],
#    'model__min_samples_leaf': [2, 4],  # [1, 2, 4]
#}

param_grid = {
    'model__n_estimators': [100, 200],  # , 300],
    'model__max_features': ['sqrt', 'log2'],
    'model__max_depth': [10],  # , 30, None],
    'model__min_samples_split': [2, 5],  # , 10],
    'model__min_samples_leaf': [2, 4],  # [1, 2, 4]
}

# GridSearchCV durchführen # opimise the model after precision


#grid_search_rf = GridSearchCV_mod(rf_pipe, param_grid, cv=5, scoring='f1', n_jobs=-1)  # Standard

grid_search_rf = GridSearchCV(rf_pipe, param_grid, cv=5, scoring='f1', n_jobs=-1)  # Standard

#grid_search_rf = GridSearchCV(rf_pipe_mod, param_grid, cv=5, scoring='f1', n_jobs=-1)  # modified

grid_search_rf.fit(features_train, target_train)

# Beste Parameter und Ergebnis anzeigen
print(f'Best parameters: {grid_search_rf.best_params_}')
print(f'Best score: {grid_search_rf.best_score_}')


In [ ]:
# predict and evaluate on test set

print(classification_report(target_test, grid_search_rf.predict(features_test)))

In [ ]:

# Log the best parameters and the best score
logger.info('GridSearchCV Results')
logger.info(f'Best parameters: {grid_search_rf.best_params_}')
logger.info(f'Best score: {grid_search_rf.best_score_}')

# Predict and evaluate on the test set
y_pred = grid_search_rf.predict(features_test)
classification_rep = classification_report(target_test, y_pred)

# Log the classification report
logger.info('Classification Report on Test Set:')
logger.info("\n" + classification_rep)

# Print to console for quick reference
print(f'Best parameters: {grid_search_rf.best_params_}')
print(f'Best score: {grid_search_rf.best_score_}')
print(classification_report(target_test, y_pred))

print("Results have been logged to 'log\model_evaluation.log'.")

In [ ]:
# Prediction with an aim-Dataset


#  Example aim-file


# Reading the CSV file into a DataFrame
X_aim = pd.read_csv('aim_test.csv')


# Cleaning data-file
X_aim = clean_data(X_aim)


display(X_aim)
y_pred_aim = grid_search_rf.predict(X_aim)
print(X_aim)
print(y_pred_aim)

# 0 is not depressive
# 1 is depressiv

In [ ]:
target_col = 'depressiveness'

print(target_col)


# Sort the values and select the top 10 features
top_10_features_series = feature_importance_rf.nlargest(10).sort_values()

# Convert the top 10 features into a list of tuples
top_10_features = list(top_10_features_series.items())

# Extract just the feature names into a list
top_10_feature_names = top_10_features_series.index.tolist()

# Print the list of top 10 feature names for verification
print("Top 10 Feature Names:", top_10_feature_names)


In [ ]:
import pickle
import pandas as pd


#  Save with pickle


with open('model_depression.pkl', 'wb') as file:
    pickle.dump(grid_search_rf, file)
    pickle.dump(target_col, file)
#    pickle.dump(top_10_feature_names, file)
    pickle.dump(all_features, file)

In [ ]:
# Prediction of one Data-Set

import pandas as pd

# Example input data
aim = [19, 33.33, 'Class I Obesity', 0, 0, 1, 1, 0, 7, 'Moderate', 11, 'Mild', 9]

# Define column names
columns = ['age', 'bmi', 'who_bmi', 'sleepiness', 'anxiety_diagnosis', 'gender', 'school_year', 
           'depression_diagnosis', 'epworth_score', 'anxiety_severity', 
           'gad_score', 'depression_severity', 'phq_score']

# Convert aim into a DataFrame
X_aim = pd.DataFrame([aim], columns=columns)



display(X_aim)
y_pred_aim = grid_search_rf.predict(X_aim)
print(X_aim)
print(y_pred_aim)

# 0 is not depressive
# 1 is depressiv
